# Práctica 2: Algoritmos metaheurísticos

## Sistemas Inteligentes

### Curso académico 2022-2023

#### Profesorado:

* Juan Carlos Alfaro Jiménez (`JuanCarlos.Alfaro@uclm.es`)
* Guillermo Tomás Fernández Martín (`Guillermo.Fernandez@uclm.es`)
* María Julia Flores Gallego (`Julia.Flores@uclm.es`)
* Ismael García Varea (`Ismael.Garcia@uclm.es`)
* Luis González Naharro (`Luis.GNaharro@uclm.es`)
* Aurora Macías Ojeda (`Profesor.AMacias@uclm.es`)
* Marina Sokolova Sokolova (`Marina.Sokolova@uclm.es`)

## 0. Preliminares

Antes de comenzar con el desarrollo de esta práctica es necesario **descargar**, en **formato `.py`**, el código de la **práctica anterior** con el nombre **`utils.py`**. Para ello, pulsamos, en la libreta de la primera práctica, **`File > Export .py`**.

Una vez hemos descargado y nombrado correctamente el fichero, lo **añadimos** al espacio de trabajo de la **libreta** de la práctica **actual** a través de **`Attached data > Notebook files > Upload files`** y subimos el fichero `utils.py` descargado en el paso anterior.

Tras esto, debemos **cambiar** el **constructor** de la clase **`Problem`** para que **reciba** directamente el **problema** a resolver, **en lugar de cargarlo desde** un **fichero**. Esto se debe a que los algoritmos metaheurísticos van a tener que resolver, en múltiples ocasiones, este problema. De esta manera nos **evitamos** la **carga computacional extra** que implica **leer** el problema desde un **fichero**. Además, también es necesario **comentar** cualquier línea de **código** que **imprima estadísticas** para evitar salidas largas.

## 1. Introducción

En esta práctica, vamos a **resolver** un **problema** de **optimización combinatoria** mediante **algoritmos metaheurísticos**. En particular, vamos a implementar **algoritmos genéticos** para abordar el **problema** de **ruteo** de **vehículos**. En este, **varios vehículos** con **capacidad limitada** deben **recoger paquetes** en **diferentes ubicaciones** y **trasladarlos** a una **sede central** de recogida.

Además, se **analizará** y **comparará** el **rendimiento** de **diferentes algoritmos genéticos** (mediante la modificación de los pasos correspondientes) en diferentes instancias del problema.

---

## 2. Descripción del problema

El concepto de mapa que vamos a utilizar es similar al de la primera práctica. Este se representa mediante un **grafo**, donde los **nodos** representan **ciudades** y los **enlaces** indican la existencia de una **carretera en ambos sentidos** entre dos ciudades. Además, los **enlaces** tienen un **peso** asociado indicando la **distancia real** entre las dos ciudades. Al mismo tiempo, se proporciona una **sede central**, que se trata de la ciudad donde se deben dejar los paquetes. A su vez, se dispone de una **flota de vehículos** con **capacidad limitada** que deben **recoger** los **paquetes** en las ciudades correspondientes y que **inicialmente** están aparcados en la **sede central**. **En caso de que a la hora de recoger un paquete se supere la capacidad del vehículo correspondiente, este debe volver a la sede central a descargar todos los paquetes, considerando el coste que implicaría volver**.

Un mapa es un problema en particular, pero diferentes paquetes, capacidades de vehículos y ubicación de la sede central pueden dar lugar a diferentes instancias del problema. Por tanto, el **objetivo** en este problema es **recoger todos los paquetes de tal manera que los vehículos recorran la menor distancia posible**.

**Con el objetivo de simplificar la práctica en evaluación continua, se asume que se cuenta con un único vehículo. No obstante, esto podría cambiar para la evaluación no continua.**

---

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica se va a proporcionar un conjunto de mapas, sobre los cuáles se debe resolver el problema de optimización combinatoria correspondiente. Es importante destacar que la **dimensionalidad** del **problema** (número de ciudades, carreteras y paquetes) puede ser **variable**, por lo que los diferentes **algoritmos genéticos** deben ser lo suficientemente **eficientes** para que puedan **resolver** los **problemas** en un **tiempo razonable**.

**Además, algunos escenarios se van a guardar para las entrevistas de prácticas, por lo que el código debe ser lo más general posible para cargarlos de manera rápida y sencilla.**

### 3.1. Entrada

Cada escenario tendrá un fichero `.json` asociado con la siguiente estructura:

```JSON
{
    "map": {
        "cities": [
            {
                "id": id_city_0,
                "name": name_city_0,
                "lat": latitude_city_0,
                "lon": longitude_city_0
            }
        ],
        "roads": [
            {
                "origin": origin_city_id,
                "destination": destination_city_id,
                "distance": road_distance
            }
        ]
    },
    "warehouse": warehouse_city_id,
    "vehicles": [
        {
            "id": id_vehicle_0,
            "capacity": capacity_vehicle_0
        }
    ]
    "parcels": [
        {
            "id": id_parcel_0,
            "city": parcel_city_id,
            "weight": weight_parcel_0
        }
    ]
}
```

Hay cuatro elementos principales en el fichero:

* `map`: Un diccionario con el mapa, cuya descripción es la misma que la de la primera práctica
* `warehouse`: Identificador de la ciudad donde se encuentra la sede central
* `vehicles`: Lista de vehículos disponibles
* `parcels`: Lista de paquetes a recoger

Por su parte, `vehicles` contiene:

* `id`: Identificador del vehículo
* `capacity`: Capacidad máxima del vehículo

Y `parcels`:

* `id`: Identificador del paquete
* `city`: Ciudad donde se encuentra el paquete
* `weight`: Peso del paquete

**Para añadir los ficheros con los problemas al espacio de trabajo se debe usar el mismo procedimiento anterior.**

---

## 4. Plan de trabajo

### 4.1. Formalización del problema

Para resolver cualquier problema de optimización combinatoria en primer lugar hay que definir como vamos a **codificar** las **soluciones** al problema. Si bien es algo que se deja a criterio propio, se plantea la siguiente pregunta, **¿cuál puede ser la mejor representación para una secuencia de paquetes a recoger?**

Se puede comprobar si la respuesta es correcta introduciéndola en la variable `answer` del siguiente fragmento de código:

In [1]:
# Third party
import hashlib

In [2]:
check_answer = lambda answer, hashed: "The answer is " + ("" if hashlib.md5(answer).hexdigest() == hashed else "in") + "correct."

In [3]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

hashed = "90d377b31e1ac26d0d10d5612ce33ccc"  # The hashed answer
print(hashed)

check_answer(answer, hashed)

90d377b31e1ac26d0d10d5612ce33ccc


'The answer is incorrect.'

### 4.2. Implementación

A continuación se proporciona la estructura de clases recomendada para resolver el problema en cuestión. Tendréis que completar las siguientes clases de acuerdo con los algoritmos estudiados en teoría. **Debéis incluir en la siguiente celda todas las librerías que vayáis a utilizar para mantener la libreta lo más organizada posible**:

In [4]:
# TODO: Import here the libraries

#### Clase `Individual`

Esta clase proporciona la **codificación** de un **individuo** de la **población**.

Los **métodos obligatorios** que se deben añadir son:

* ``__init__(self, num_genes, generation_type, crossover_type, mutation_type)``: Inicializa el **número** de **genes** del **individuo** y el **tipo** de **operación** de **generación**, **cruce** y **mutación**. Ademas, genera la **solución** que **representa** el **individuo**.
* ``generate(num_genes, generation_type)``: Método estático para **generar** una **solución** del tamaño proporcionado de acuerdo con el tipo de operación de generación.
* ``crossover(self, individual)``: **Cruza** el **individuo actual** con el **individuo** de **entrada** de acuerdo con el tipo de operación de cruce.
* ``mutation(self)``: **Muta** el **individuo** de acuerdo con el tipo de operación de mutación.
* ``evaluate(self, problem)``: **Evalua** el **individuo** usando el **problema** a **resolver**.

Y los **métodos recomendados** son:

* ``__str__(self)``: **Representación** en formato de **cadena** de **caracteres** de un **individuo**. Método útil para depurar una lista de individuos.
* ``__repr__(self)``: **Método** invocado cuando se ejecuta **``print``** sobre el **individuo**. Método útil para depurar un solo individuo.

In [5]:
class Individual:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor

    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    # =============================================================================
    # Recommended methods
    # =============================================================================
    
    # TODO: Code here the recommended methods

    pass

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [6]:
# TODO: Test here the methods to generate a solution

In [7]:
# TODO: Test here the methods to cross individuals

In [8]:
# TODO: Test here the methods to mutate an individual

#### Clase `Genetic`

Esta clase implementa un **esquema** básico de **algoritmo genético**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)``: Inicializa el **tamaño** de la **población**, el **tipo** de **operación** de **selección**, **cruce**, y **mutación**, así como la **probabilidad** de aplicar las operaciones de **cruce** y **mutación**. Además, también inicializa el **número** de **mejores soluciones** de la **población actual** que se **mantienen** en la **siguiente población** y una **semilla** para garantizar que los **experimentos** son **reproducibles**. **Nótese que puede ser necesario añadir más argumentos si así se requiere**.
* ``def __call__(self, problem)``: **Método** que se **ejecuta** cuando se llama a un **objeto** de la **clase como** si fuese una **función**. En este **método** se debe **implementar** el **esquema básico** de un **algoritmo genético** que se encargue de ejecutar los pasos correspondientes. 
* ``def generate_population(self, problem)``: **Genera** la **población inicial** de acuerdo con el **problema** a resolver.
* ``def select_population(self, population, scores)``: **Selecciona** los **padres** a utilizar para la operación de cruce.
* ``def crossover(self, population)``: **Cruza pares** de **padres** teniendo en cuenta la probabilidad de cruce.
* ``def mutation(self, population)``: **Muta** los **individuos cruzados** teniendo en cuenta la probabilidad de mutación.
* ``def evaluate(self, population, problem)``: **Evalua** los **nuevos individuos** de acuerdo con el problema a resolver.
* ``def combine(self, population)``: **Forma** la **nueva generación** de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.

In [9]:
class Genetic:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor

    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    pass

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [10]:
# TODO: Test here the methods to generate a population

In [11]:
# TODO: Test here the methods to select individuals

In [12]:
# TODO: Test here the methods to combine populations

#### Clase `CVRP`

Esta clase representa el problema en cuestión, esto es, el **problema** de **ruteo** de **vehículos**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, filename, algorithm)``: Inicializa el **problema** en cuestión y el **algoritmo** a usar para resolverlo. A su vez, se debe crear un **diccionario** que contenga como **clave** un **identificador** de **paquete** y como **valor** una **tupla** con la **ciudad** donde se encuentra dicho paquete y su **peso**.
* ``def __call__(self)``: **Resuelve** el **problema** en cuestión.
* ``def evaluate(self, solution)``: **Evalua** una **solución** para el **problema** en cuestión, **teniendo en cuenta** las **restricciones correspondientes**.
* ``def search(self, departure, goal)``: **Resuelve** un **problema** de **búsqueda** de **caminos** dada las **ciudades** de **salida** y **meta**.

**Nótese que se puede crear una estructura de datos para agilizar el proceso de búsqueda de caminos requerido por el algoritmo ¿cuál puede ser?**

In [13]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

encoded = "0fea6a13c52b4d4725368f24b045ca84"  # The hashed answer
print(encoded)

check_answer(answer, encoded)

0fea6a13c52b4d4725368f24b045ca84


'The answer is incorrect.'

In [14]:
class CVRP:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor

    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    pass

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [15]:
# TODO: Test here the method to initialize the capacited vehicle routing problem

In [16]:
# TODO: Test here the method to solve the search problem

In [17]:
# TODO: Test here the method to solve the capacited vehicle routing problem

### 4.3. Estudio y mejora de los algoritmos

Una vez que los algoritmos han sido implementados, se debe **estudiar** su **rendimiento**. Para ello, se debe comparar la **calidad** de las **soluciones obtenidas**, así como las **diferentes estadísticas** que se consideren adecuadas (número de generaciones, tiempo de ejecución, etc.). Factores como el tamaño máximo del problema que se soporta sin un error de memoria, así como el efecto temporal de usar escenarios más complejos son otros factores a tener en cuenta. Además, se **pueden proponer** y se valorarán la incorporación de **técnicas** que **permitan acelerar** la **ejecución** de los **algoritmos**.

---

In [18]:
# TODO: Experiment here with the small problem

In [19]:
# TODO: Experiment here with the medium problem

In [20]:
# TODO: Experiment here with the large problem

### 5. Entrega y evaluación

Al igual que la práctica anterior, esta se debe **hacer en pares**. No obstante, en **casos excepcionales** se permite realizarla **individualmente**. **La fecha límite para subir la práctica es el 18 de diciembre de 2022 a las 23:55**. Las **entrevistas y evaluaciones** se realizarán la **semana siguiente**.

Algunas consideraciones:

* **En caso de que no se haya entregado la primera práctica, o se haya sacado menos de un cuatro, se podrán entregar conjuntamente en esta fecha. No obstante, se considerará únicamente un 90% de la nota global de prácticas**.
* Está práctica supone el **70%** de la **nota** en este apartado.
* La práctica se **evaluará** mediante una **entrevista individual** con el profesorado. Las fechas de las entrevistas se publicarán con antelación.
*  Se proporcionará un **conjunto** de **casos** de **prueba preliminares** (varios mapas e instancias) que se **deben resolver correctamente**. En caso contrario, la práctica se considerará suspensa.
* La **entrevista** consistirá en una serie de **preguntas** acerca del **código**.

**Por último, para la evaluación no continua se requirirá la implementación del algoritmo de búsqueda por ascenso de colinas. Además, este se deberá utilizar para inicializar la población del algoritmo genético, en lugar de que sea aleatoria.**